## Importar librerías

In [27]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 5616 Features: 296
Sell -> Trades: 4726 Features: 296


In [29]:
df_buy

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,0.630078,-0.562533,3.786181,-0.000168,0.000453,168.0,-0.000561,0.000457,116.0,-0.000673,...,98.0,-0.000505,0.000468,110.0,-0.000561,0.000501,55.0,0.625000,0.333333,0.0
1,-0.424378,0.740729,4.827235,-0.000812,0.000680,257.0,-0.001106,0.000664,173.0,-0.000885,...,100.0,0.001182,0.000623,121.0,0.001108,0.000633,99.0,0.625000,0.166667,1.0
2,-0.948941,-0.098797,6.832373,-0.001059,0.000710,202.0,-0.001059,0.000669,111.0,-0.000988,...,171.0,-0.001270,0.000961,238.0,-0.002257,0.001027,183.0,0.833333,0.333333,1.0
3,-0.931600,1.480015,2.741621,-0.000192,0.000436,70.0,-0.000447,0.000441,64.0,-0.000703,...,30.0,0.001209,0.000292,27.0,0.000761,0.000251,33.0,0.916667,0.333333,1.0
4,-0.354712,-0.491995,4.765011,-0.000383,0.000717,49.0,-0.000612,0.000711,39.0,-0.000612,...,17.0,-0.001148,0.000489,35.0,-0.001339,0.000476,42.0,0.333333,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,0.192469,0.583276,0.611250,-0.000055,0.000139,12.0,-0.000055,0.000143,4.0,-0.000055,...,15.0,-0.000110,0.000223,4.0,-0.000055,0.000246,10.0,0.250000,0.500000,1.0
5612,-1.296178,0.917863,2.487331,-0.000267,0.000189,58.0,-0.000386,0.000172,35.0,-0.000356,...,19.0,-0.000504,0.000110,24.0,-0.000445,0.000110,26.0,0.333333,0.333333,0.0
5613,-0.760829,0.093952,12.718637,-0.001641,0.001253,348.0,-0.002035,0.001129,321.0,-0.002100,...,600.0,-0.000263,0.000632,131.0,-0.000197,0.000600,53.0,0.416667,0.666667,1.0
5614,-0.508773,-0.467232,18.286138,-0.003639,0.002373,449.0,-0.004153,0.002235,369.0,-0.005328,...,149.0,-0.005394,0.001800,167.0,-0.005114,0.001675,130.0,0.416667,0.500000,1.0


In [30]:
df_sell

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,0.720993,-0.182869,4.878750,0.000562,0.000386,80.0,0.000749,0.000333,55.0,0.000749,...,27.0,-0.000916,0.000194,26.0,-0.001135,0.000180,35.0,0.375000,0.166667,1.0
1,0.399543,-0.801292,4.013028,0.000450,0.000427,65.0,0.000578,0.000418,33.0,0.000643,...,48.0,0.000578,0.000457,50.0,0.000900,0.000485,63.0,0.250000,0.166667,0.0
2,0.141719,-0.160043,15.040604,0.001234,0.002063,319.0,0.001689,0.002019,345.0,0.001364,...,541.0,0.005727,0.001990,515.0,0.006054,0.001874,422.0,0.750000,0.666667,0.0
3,-0.751806,0.541827,11.133802,0.000439,0.001170,272.0,0.001208,0.001155,237.0,0.001922,...,449.0,0.003204,0.000947,843.0,0.003298,0.000886,119.0,0.666667,0.666667,1.0
4,0.421038,-0.975665,5.032996,0.000649,0.000453,95.0,0.000840,0.000418,57.0,0.001108,...,71.0,0.000726,0.000726,57.0,0.001376,0.000726,85.0,0.208333,0.166667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4721,0.423245,1.757368,10.276026,0.001599,0.002508,66.0,0.002112,0.002505,117.0,0.004128,...,102.0,0.000547,0.002834,235.0,-0.002114,0.002819,146.0,0.333333,0.666667,1.0
4722,0.089368,-0.737972,9.988488,0.000918,0.001291,253.0,0.001342,0.001307,338.0,0.002544,...,241.0,0.000000,0.001350,282.0,-0.000973,0.001415,252.0,0.625000,0.500000,1.0
4723,-1.125035,2.403532,4.971833,0.000471,0.001489,227.0,0.000737,0.001456,259.0,0.001503,...,88.0,0.003726,0.000929,67.0,0.004392,0.000854,121.0,0.375000,0.166667,0.0
4724,0.554286,-0.514150,6.108529,0.000780,0.000832,173.0,0.000947,0.000853,172.0,0.000668,...,196.0,-0.000195,0.001071,257.0,0.000251,0.001135,225.0,0.500000,0.666667,1.0


In [31]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [32]:
# Dividir 
# Preparación de los datos de compra
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train = X_buy
y_buy_train = y_buy
# Preparación de los datos de venta
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train = X_sell
y_sell_train = y_sell

In [33]:
param_grid = {
    'xgb__n_estimators': [50, 100, 200, 500],
    'xgb__max_depth': [3, 5, 7, 9],
    'xgb__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'xgb__subsample': [0.6, 0.8, 1.0],
    'xgb__colsample_bytree': [0.6, 0.8, 1.0],
    'xgb__gamma': [0, 0.1, 0.2, 0.5],
    'xgb__min_child_weight': [1, 3, 5],
    'xgb__reg_alpha': [0, 0.01, 0.1, 1],
    'xgb__reg_lambda': [1, 1.5, 2]
}

In [34]:
# Crear pipe y entrenar
from sklearn.decomposition import PCA
pipe_buy = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_buy = GridSearchCV(
    estimator=pipe_buy,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_buy.fit(X_buy_train, y_buy_train)
model_buy = grid_search_buy.best_estimator_
pipe_sell = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_sell = GridSearchCV(
    estimator=pipe_sell,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_sell.fit(X_sell_train, y_sell_train)
model_sell = grid_search_sell.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [35]:
grid_search_buy.best_score_

0.5651739906136017

In [36]:
grid_search_sell.best_score_

0.5442209470116446

In [37]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [38]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())